In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import numpy as np

In [2]:
# Sample dataset: user interactions with posts
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3],
    'post_id': [101, 102, 103, 101, 104, 103, 105],
    'interaction': [1, 1, 0, 1, 1, 0, 1],  # 1: interacted, 0: not interested
    'post_content': [
        "I love this new feature",
        "This is a terrible update",
        "Amazing picture of the sunset",
        "I love this new feature",
        "Great improvements in the design",
        "Amazing picture of the sunset",
        "Nice work, keep it up!"
    ]
}

In [3]:
df = pd.DataFrame(data)

In [4]:
# Collaborative Filtering: Creating a user-post interaction matrix
interaction_matrix = df.pivot_table(
    index='user_id', columns='post_id', values='interaction').fillna(0)

In [5]:
# Cosine similarity between users
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

In [6]:
# Get recommendations for a specific user (e.g., user 1)
def get_collaborative_recommendations(user_id, interaction_matrix, user_similarity_df, top_n=2):
    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[
        1:]
    similar_user_id = similar_users.index[0]

    # Find posts the similar user has interacted with
    similar_user_posts = interaction_matrix.loc[similar_user_id]

    # Recommend posts not interacted by current user
    user_posts = interaction_matrix.loc[user_id]
    recommended_posts = similar_user_posts[(
        similar_user_posts > 0) & (user_posts == 0)].index.tolist()

    return recommended_posts[:top_n]

In [7]:
# Sentiment Analysis: Filter recommendations based on sentiment
def analyze_sentiment(post_content):
    sentiment = TextBlob(post_content).sentiment.polarity
    return sentiment

In [8]:
def filter_by_sentiment(recommended_posts, df, sentiment_threshold=0.0):
    filtered_posts = []
    for post_id in recommended_posts:
        post_content = df[df['post_id'] == post_id]['post_content'].values[0]
        sentiment = analyze_sentiment(post_content)
        if sentiment >= sentiment_threshold:  # Only recommend positive or neutral posts
            filtered_posts.append(post_id)
    return filtered_posts

In [9]:
# Example: Get recommendations for user 1
user_id = 1
collab_recommendations = get_collaborative_recommendations(
    user_id, interaction_matrix, user_similarity_df)
print(
    f"Collaborative Filtering Recommendations for User {user_id}: {collab_recommendations}")

Collaborative Filtering Recommendations for User 1: [104]


In [10]:
# Apply sentiment analysis filter
final_recommendations = filter_by_sentiment(collab_recommendations, df)
print(
    f"Final Recommendations after Sentiment Analysis for User {user_id}: {final_recommendations}")

Final Recommendations after Sentiment Analysis for User 1: [104]
